In [1]:
import JDM
import pandas as pd

In [2]:
t_search = JDM.Trie() 

def build_mwe(): 
    df = pd.read_csv("MWE/mwe_id_word.csv", sep=";")

    from alive_progress import alive_bar

    with alive_bar(len(df.index), force_tty=None) as bar:
        for index, row in df.iterrows():
            t_search.addword(row['words'].split())
            
            bar()

In [3]:
build_mwe()

|████████████████████████████████████████| 1738444/1738444 [100%] in 1:26.9 (19993.74/s) 


In [39]:
def remove_useless_words(phrase):
    # Enlever la majuscule au début
    phrase = phrase[0].lower() + phrase[1:]

    # Enlever les articles
    phrase = phrase.replace("le", "")
    phrase = phrase.replace("la", "")
    phrase = phrase.replace("les", "")
    phrase = phrase.replace("un", "")
    phrase = phrase.replace("une", "")
    phrase = phrase.replace("des", "")
    phrase = phrase.replace("du", "")
    phrase = phrase.replace("de", "")

    phrase = phrase.replace("  ", " ")


    return phrase

def found_compound_terms(phrase): 

    #print(t_search.isCT([phrase[0], phrase[1]])) 

    compound_term = []
    phrase_without_compound_terms = []

    for i in range(len(phrase)):
        print("for i : ", i)
        compound_term = []

        while i < len(phrase) and t_search.isCT(compound_term + [phrase[i]]): 
            print("while i : ", i)
            compound_term += [phrase[i]]
            i = i + 1

        if len(compound_term) > 0: 
            # Transformer le tableau en string et l'ajouter à phrase_without_compound_terms
            phrase_without_compound_terms += [''.join(compound_term)]
        else : 
            # On ajout phrase[i] à phrase_without_compound_terms
            phrase_without_compound_terms += [phrase[i]]



        #print(i, phrase[i])

    return phrase_without_compound_terms

def analyser(phrase): 
    # Transformer la phrase en tableau
    phrase = phrase.split(" ")

    # Trouver les termes composées
    phrase = found_compound_terms(phrase)

    


    print(phrase)

In [40]:
analyser("Le chat blanc boit du lait de chèvre")

True
for i :  0
while i :  0
while i :  1
for i :  1
while i :  1
while i :  2
for i :  2
while i :  2
for i :  3
while i :  3
for i :  4
while i :  4
while i :  5
for i :  5
while i :  5
while i :  6
while i :  7
for i :  6
while i :  6
while i :  7
for i :  7
while i :  7
['Lechat', 'chatblanc', 'blanc', 'boit', 'dulait', 'laitdechèvre', 'dechèvre', 'chèvre']
